# W2D1 Tutorial 3: Extreme value analysis - the GEV distribution

#### __Week 2, Day 4, Extremes & Vulnerability__
##### __Content creators:__ Matthias Aengenheyster, Joeri Reinders
##### __Content reviewers:__ TBD
##### __Content editors:__ TBD
##### __Production editors:__ TBD
##### __Our 2023 Sponsors:__ TBD

## Tutorial Objectives:

In the previous tutorial we computed "empirical return levels". The second method to determine return values is by computing the probability of exceedence from a probability
density function that has been fitted to the data. For example the normal distribution pdf that we created in
tutorial 1. However we noticed that the normal distribution did not fit well to our precipitation data.

By the end of the tutorial you will be able to:
- Create the Quantile plot to assess how well a distribution fits the data
- Fit a GEV distribution to the data
- Understand how the parameters of the GEV distribution control its behavior.

## Setup

In [ ]:
# Installs

In [ ]:
#install dependencies - taken from <Yosmely Bermúdez> comments for Tutorial 6
# We need this to install eigen which is needed for SDFC to install correctly
# !pip install -q condacolab
# import condacolab
# condacolab.install()
# !conda install eigen
# !pip install -v https://github.com/yrobink/SDFC/archive/master.zip#subdirectory=python
# !pip install https://github.com/njleach/mystatsfunctions/archive/master.zip
# !pip install numpy
# !pip install matplotlib
# !pip install seaborn
# !pip install pandas
# !pip install cartopy
# !pip install scipy
# !pip install texttable

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats
gev = stats.genextreme

In [ ]:
import extremes_functions as ef
from mystatsfunctions import OLSE, LMoments
import SDFC as sd

Lets repeat our first steps from tutorials 1 and 2: 
1) open the precipitation annual block maxima record, 
2) create a histogram of the data and plot the a normal distribution probability density function.

In [ ]:
import os, pooch

fname = 'precipitationGermany_1920-2022.csv'
if not os.path.exists(fname):
    url = "https://osf.io/xs7h6/download"
    fname = pooch.retrieve(url, known_hash=None)

data = pd.read_csv(fname, index_col=0).set_index('years')
data.columns=['precipitation']
precipitation = data.precipitation

In [ ]:
bins=np.arange(0,precipitation.max(),2)
sns.histplot(precipitation,bins=bins,stat='density')
plt.xlim(bins[0],bins[-1])

ylim = plt.ylim()

x = np.arange(0,100,1)
plt.plot(x,stats.norm.pdf(x,precipitation.mean(),precipitation.std()),c='k',lw=3)

Next we create a Quantile plots (QQ plot) from where we judge the fit of the our data to a normal distribution. Recall from the lecture that a QQ plot plots the *actual* percentiles of the data against the *expected* percentiles assuming the distribution is e.g. a normal distribution.

In [ ]:
x = np.linspace(0,1,100)

plt.plot(
    stats.norm.ppf(x,precipitation.mean(), precipitation.std()),
    np.quantile(precipitation,x),
    'o')
plt.plot([0,100],[0,100],'k')

plt.xlim(10,72)
plt.ylim(10,72)

plt.xlabel('Normal Quantiles')
plt.ylabel('Sample Quantiles')

plt.grid(True)
plt.gca().set_aspect('equal')

Hopefully you agree with me that the fit can be better. Especially the extreme values seem to be under- 
and overestimated by the normal distribution model.So let’s try something new... because there are many other 
distributions. For example the General Extreme Value distribution.

The GEV distribution has three parameters that define its shape: the location, scale and shape parameter. 
Just to give you some context, the normal distribution only has two parameters: the mean and the standard 
deviation. If you increase the mean the distribution will shift to higher values, and if you increase the 
standard deviation the distribution will become wider. The normal distribution therefore is symmetrical as 
there is no parameter that affects its skew – this often makes the normal distribution problematic to use. 

In the GEV distribution the location and scale parameter behave similar to the mean and standard deviation
in the normal distribution. The shape-parameter affects the tails of the distribution by making them thinner 
or thicker. Extreme event distributions often have thick tails meaning that they are slightly skewed. Adjusting
the shape-parameter, thus affects the skew (and kurtosis) of your data.

To fit the parameters of the GEV distribution we use the SDFC package. The normal distribution has the parameters mean and standard deviation. In contrast, the the GEV 
distribution has three parameters (location,scale,shape) – these also depend on each other so they cannot be estimated individually.
Fitting the parameters to the data is done by the package.

In [ ]:
fit, model = ef.fit_return_levels_sdfc(precipitation.values,times=np.arange(1.1,1000),periods_per_year=1,kind='GEV',N_boot=10,full=True,model=True)

The fit functions returns two objects, here called "fit" and "model". "fit" is a data container (an xarray.Dataset) that contains the results of the fitting procedure: the return periods, bootstrapping samples, empirical return periods and levels (called return_period_obs and return_level_obs), and fitting estimates for the GEV parameters and return levels.

"model" is a Class, that allows us the access various pieces of information. Just running it prints a neat description of the result of the fitting procedure:

In [ ]:
fit

In [ ]:
model

Let's plot again the histogram of the data, and on top the GEV distribution with the fitted parameters:

In [ ]:
bins=np.arange(0,precipitation.max(),2)
sns.histplot(precipitation,bins=bins,stat='density')
plt.xlim(bins[0],bins[-1])


x = np.arange(80)
plt.plot(x,gev.pdf(x, -fit.xi.mean().values, loc=fit.mu.mean().values, scale=fit.sigma.mean().values),'k',lw=3)

And also create a QQ-plot:

In [ ]:
x = np.linspace(0,1,100)

plt.plot(
    gev.ppf(x, -fit.xi.mean().values, loc=fit.mu.mean().values, scale=fit.sigma.mean().values), 
    np.quantile(precipitation,x),'o')

plt.plot([0,100],[0,100],'k')

plt.xlim(10,72)
plt.ylim(10,72)

plt.xlabel('GEV Quantiles')
plt.ylabel('Sample Quantiles')

plt.grid(True)
plt.gca().set_aspect('equal')

This looks much better! Apparently the GEV distribution is a better fit for the data than the Normal distribution

Now create a plot with two panels each one showing a QQplot (one of the normal distribution fit, one of the GEV 
distribution fit). Similarly create a plot with two panels that show the histogram and pdf (again one with the normal 
distribution pdf and one with the GEV distribution pdf). 

Finally, plot both pdf's (normal and GEV) over each other so you can see the differences between them.

In [ ]:
fig, ax = plt.subplots(1,2,sharey=True,sharex=True,constrained_layout=True)

x = np.linspace(0,1,100)

# Normal
ax[0].plot(
    stats.norm.ppf(x,precipitation.mean(), precipitation.std()),
    np.quantile(precipitation,x),
    'o')
ax[0].plot([0,100],[0,100],'k')

ax[0].set_xlabel('Standard Normal Quantiles')
ax[0].set_ylabel('Sample Quantiles')

ax[0].grid(True)

# GEV
ax[1].plot(
    gev.ppf(x, -fit.xi.mean().values, loc=fit.mu.mean().values, scale=fit.sigma.mean().values), 
    np.quantile(precipitation,x),
    'o')

ax[1].plot([0,100],[0,100],'k')

ax[1].set_xlim(10,72)
ax[1].set_ylim(10,72)

ax[0].set_aspect('equal')
ax[1].set_aspect('equal')

ax[1].grid(True)
ax[1].set_xlabel('GEV quantiles')

ax[0].set_title('Normal distribution fit')
ax[1].set_title('GEV distribution fit')

In [ ]:
fig, ax = plt.subplots(1,2,sharex=True,sharey=True,constrained_layout=True)
sns.histplot(precipitation,ax=ax[0],bins=np.arange(0,80,2),stat='density')
sns.histplot(precipitation,ax=ax[1],bins=np.arange(0,80,2),stat='density')

x = np.arange(0,100,1)
ax[0].plot(x,stats.norm.pdf(x,precipitation.mean(),precipitation.std()),c='k',lw=3)
ax[1].plot(x,gev.pdf(x, -fit.xi.mean().values, loc=fit.mu.mean().values, scale=fit.sigma.mean().values),c='k',lw=3)


ax[0].set_title('Normal')
ax[1].set_title('GEV')

In [ ]:
plt.plot(x,stats.norm.pdf(x,precipitation.mean(),precipitation.std()),c='C3',lw=3,label='Normal')
plt.plot(x,gev.pdf(x, -fit.xi.mean().values, loc=fit.mu.mean().values, scale=fit.sigma.mean().values),c='k',lw=3,label='GEV')
plt.legend()
plt.xlabel('precipitation (mm/day)')
plt.ylabel('Density')

How well do the two fitted distributions reflect the observed data and how do they compare to each other? 

## Coding excercise: Parameters of the GEV distribution

Play a little with the gev.pdf function to get a better sense of how the parameters affect the shape of the pdf (distribution). Plot the distribution against the
‘x’ sequence and randomly change the parameters. What does each parameter do? 

Create three plots. In each, one of the parameters (location, scale, shape) is varied while the other two are held constant. The parameter values and ranges below may be a useful starting point.

In [ ]:
loc = 26
scale = 7
shape = 0

range_loc = np.arange(20,40,4)
range_scale = np.arange(4,10,1)
range_shape = np.arange(-0.4,0.4+0.1,0.1)

# useful variable range for the chosen parameters
x = np.linspace(0,80,1000)


### student

In [ ]:
...

### solution

In [ ]:
fig, ax = plt.subplots()
x = np.linspace(0,80,1000)
colors = plt.cm.viridis(np.linspace(0,1,range_loc.size))
i = 0
for loci in range_loc:
    p = plt.plot(x,gev.pdf(x, -shape, loc=loci, scale=scale),color=colors[i],label='loc = %i' % loci)
    i += 1
plt.legend()

plt.xlabel('precipitation (mm/day)')
plt.ylabel('Density')

In [ ]:
fig, ax = plt.subplots()
x = np.linspace(0,80,1000)
colors = plt.cm.viridis(np.linspace(0,1,range_scale.size))
i = 0
for scalei in range_scale:
    plt.plot(x,gev.pdf(x, -shape, loc=loc, scale=scalei),color=colors[i],label='scale = %.2f' % scalei)
    i += 1
plt.legend()

plt.xlabel('precipitation (mm/day)')
plt.ylabel('Density')

In [ ]:
fig, ax = plt.subplots()
x = np.linspace(0,80,1000)
colors=plt.cm.viridis(np.linspace(0,1,range_shape.size))
i = 0
for shapei in range_shape:
    plt.plot(x,gev.pdf(x, -shapei, loc=loc, scale=scale),color=colors[i],label='shape = %.2f' % shapei)
    i += 1
plt.legend()

plt.xlabel('precipitation (mm/day)')
plt.ylabel('Density')

## Question: How do the three parameters impact the shape of the distribution? What can you think of how these parameters affect extreme events?